In [75]:
from dotenv import load_dotenv
import os
import mysql.connector

load_dotenv()


True

In [70]:
db = mysql.connector.connect(
    host= os.getenv('DB_ENDPOINT'),      # e.g., localhost or your DB server address
    user= os.getenv("DB_USER"),  # your MySQL username
    password= os.getenv("DB_PASSWORD"),  # your MySQL password
    database= os.getenv("DB_NAME")  # the database where you want to import data
)

cursor = db.cursor()

In [ ]:
db = mysql.connector.connect(
    host= os.getenv('DB_ENDPOINT'),      # e.g., localhost or your DB server address
    user= os.getenv("DB_USER"),  # your MySQL username
    password= os.getenv("DB_PASSWORD"),  # your MySQL password
    database= os.getenv("DB_NAME")  # the database where you want to import data
)

cursor = db.cursor()

In [77]:
import pandas as pd
import numpy as np
df = pd.read_csv('cleanish.csv')

In [144]:
#creating df_team to match table format
df_team = pd.concat([df['team'], df['Opponent']]).unique()
df_team = pd.DataFrame(df_team).reset_index(drop=True)
df_team.insert(0, 'team_id', range(1, len(df_team)+1))
df_team.columns = ['team_id', 'team_name']
df_team.insert(2, 'team_home_venue', None)
df_team.nunique()

team_id            23
team_name          23
team_home_venue     0
dtype: int64

In [145]:
insert_query_teams = """
INSERT INTO team(team_name, team_home_venue)
VALUES (%s, %s)
"""

for index, row in df_team.iterrows():
    values = (row['team_name'], row['team_home_venue'])
    cursor.execute(insert_query_teams,values)
    
db.commit()

In [146]:
df_game = df[['year', 'round', 'team', 'Opponent']]
df_game.insert(4, 'venue', None)
df_game.insert(0, 'game_id', range(1, len(df_game)+1))

#replace team and opponent with its team_ids
team_id_dict = pd.Series(df_team.team_id.values, index=df_team.team_name).to_dict()
df_game['team1_id'] = df_game['team'].map(team_id_dict)
df_game['team2_id'] = df_game['Opponent'].map(team_id_dict)

df_game.isnull().sum()

C:\Users\weezi\AppData\Local\Temp\ipykernel_5740\3841568468.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_game['team1_id'] = df_game['team'].map(team_id_dict)
C:\Users\weezi\AppData\Local\Temp\ipykernel_5740\3841568468.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_game['team2_id'] = df_game['Opponent'].map(team_id_dict)


game_id        0
year           0
round          0
team           0
Opponent       0
venue       9052
team1_id       0
team2_id       0
dtype: int64

In [147]:
insert_query_game = """
    INSERT INTO game (year, round_no, venue, team1_id, team2_id)
    VALUES (%s, %s, %s, %s, %s)
    """

values = list(df_game[['year', 'round', 'venue', 'team1_id', 'team2_id']].itertuples(index=False, name=None))
cursor.executemany(insert_query_game, values)

db.commit()

In [148]:
df.insert(0, 'game_id', range(1, len(df)+1))

df_gameteam = pd.DataFrame(columns=[
    'game_id', 'team_name', 'is_home_team', 'is_winner', 'score', 'goals', 'behinds', 
    'disposals', 'kicks', 'marks', 'handballs', 'hit_outs', 'tackles', 'rebound_50s', 
    'inside_50s', 'clearances', 'clangers', 'freekicks_for', 'freekicks_against', 
    'brownlow_votes', 'contested_possessions', 'uncontested_possessions', 
    'marks_inside_50s', 'one_percenters', 'bounces', 'goal_assist'
])

for index, row in df.iterrows():
    team1_row = {
        'game_id': row['game_id'],
        'team_name': row['team'],
        'is_home_team': None,  # Assuming team1 is home, adjust if needed
        'is_winner': 1 if row['score_team1'] > row['score_team2'] else 0,
        'score': row['score_team1'],
        'goals': row['goals_team1'],
        'behinds': row['behinds_team1'],
        'disposals': row['disposals_team1'],
        'kicks': row['kicks_team1'],
        'marks': row['marks_team1'],
        'handballs': row['handballs_team1'],
        'hit_outs': row['hit_outs_team1'],
        'tackles': row['tackles_team1'],
        'rebound_50s': row['rebound_50s_team1'],
        'inside_50s': row['inside_50s_team1'],
        'clearances': row['clearances_team1'],
        'clangers': row['clangers_team1'],
        'freekicks_for': row['freekicks_for_team1'],
        'freekicks_against': row['freekicks_agains_team1'],
        'brownlow_votes': row['brownlow_votes_team1'],
        'contested_possessions': row['contested_possesions_team1'],
        'uncontested_possessions': row['uncontested_possesions_team1'],
        'marks_inside_50s': row['marks_inside_50_team1'],
        'one_percenters': row['one_percenters_team1'],
        'bounces': row['bounces_team1'],
        'goal_assist': row['goal_assist_team1']
    }

    team2_row = {
        'game_id': row['game_id'],
        'team_name': row['Opponent'],
        'is_home_team': None,  # Assuming team2 is away
        'is_winner': 1 if row['score_team2'] > row['score_team1'] else 0,
        'score': row['score_team2'],
        'goals': row['goals_team2'],
        'behinds': row['behinds_team2'],
        'disposals': row['disposals_team2'],
        'kicks': row['kicks_team2'],
        'marks': row['marks_team2'],
        'handballs': row['handballs_team2'],
        'hit_outs': row['hit_outs_team2'],
        'tackles': row['tackles_team2'],
        'rebound_50s': row['rebound_50s_team2'],
        'inside_50s': row['inside_50s_team2'],
        'clearances': row['clearances_team2'],
        'clangers': row['clangers_team2'],
        'freekicks_for': row['freekicks_for_team2'],
        'freekicks_against': row['freekicks_agains_team2'],
        'brownlow_votes': row['brownlow_votes_team2'],
        'contested_possessions': row['contested_possesions_team2'],
        'uncontested_possessions': row['uncontested_possesions_team2'],
        'marks_inside_50s': row['marks_inside_50_team2'],
        'one_percenters': row['one_percenters_team2'],
        'bounces': row['bounces_team2'],
        'goal_assist': row['goal_assist_team2']
    }

    df_gameteam = pd.concat([df_gameteam, pd.DataFrame([team1_row, team2_row])], ignore_index=True)

# Check the result
df_gameteam['team_id'] = df_gameteam['team_name'].map(team_id_dict)
df_gameteam.insert(0, 'game_team_id', range(1, len(df_gameteam)+1))


C:\Users\weezi\AppData\Local\Temp\ipykernel_5740\313953048.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_gameteam = pd.concat([df_gameteam, pd.DataFrame([team1_row, team2_row])], ignore_index=True)


In [165]:
df_gameteam = df_gameteam.astype(object).where(pd.notna(df_gameteam), None)
df_gameteam

,game_team_id,game_id,team_name,is_home_team,is_winner,score,goals,behinds,disposals,kicks,...,freekicks_for,freekicks_against,brownlow_votes,contested_possessions,uncontested_possessions,marks_inside_50s,one_percenters,bounces,goal_assist,team_id
0,1,1,Essendon,None,1,103.0,14,19.0,262,234.0,...,29.0,31.0,None,None,None,None,None,None,None,1
1,2,1,St Kilda,None,0,69.0,9,15.0,224,203.0,...,31.0,29.0,None,None,None,None,None,None,None,10
2,3,2,Collingwood,None,0,42.0,6,6.0,236,209.0,...,30.0,23.0,None,None,None,None,None,None,None,2
3,4,2,Essendon,None,1,96.0,14,12.0,249,225.0,...,23.0,30.0,None,None,None,None,None,None,None,1
4,5,3,Carlton,None,1,91.0,12,19.0,242,199.0,...,33.0,28.0,None,None,None,None,None,None,None,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18099,18100,9050,Sydney,None,1,76.0,11,10.0,336,211.0,...,27.0,20.0,3.0,146.0,175.0,9.0,65.0,0.0,5.0,12
18100,18101,9051,Brisbane Lions,None,0,78.0,11,12.0,329,220.0,...,31.0,22.0,None,155.0,170.0,11.0,57.0,3.0,9.0,17
18101,18102,9051,Greater Western Sydney,None,1,80.0,12,8.0,310,188.0,...,22.0,31.0,None,131.0,172.0,10.0,63.0,5.0,7.0,21
18102,18103,9052,Geelong,None,1,85.0,13,7.0,371,223.0,...,17.0,25.0,None,151.0,220.0,17.0,43.0,4.0,6.0,4


In [167]:
insert_query_gameteam = """
INSERT INTO game_team (game_id, team_id, is_home_team, is_winner, score, goals, behinds, disposals, 
    kicks, marks, handballs, hit_outs, tackles, rebound_50s, inside_50s, clearances, 
    clangers, freekicks_for, freekicks_against, brownlow_votes, contested_possessions, 
    uncontested_possessions, marks_inside_50s, one_percenters, bounces, goal_assist)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
values = list(df_gameteam[['game_id', 'team_id', 'is_home_team', 'is_winner', 'score', 'goals', 'behinds', 'disposals', 
    'kicks', 'marks', 'handballs', 'hit_outs', 'tackles', 'rebound_50s', 'inside_50s', 'clearances', 
    'clangers', 'freekicks_for', 'freekicks_against', 'brownlow_votes', 'contested_possessions', 
    'uncontested_possessions', 'marks_inside_50s', 'one_percenters', 'bounces', 'goal_assist']].itertuples(index=False, name=None))

values
cursor.executemany(insert_query_gameteam, values)

db.commit()

In [168]:
cursor.close()
db.close()